In [1]:
from typing import List
import os
import re
import random
import time

import pandas as pd
from pandas import DataFrame
import requests

import os

from util.widgets import generate_ym, generate_ymd, generate_ymd_HMS, generate_i

ym = generate_ym()
g = 'cargohose_herren'
bd = f'{g}/{ym}'
bd_a = f'{bd}/a'
b_d = f'{g}_{ym}'

os.makedirs(bd, exist_ok=True)
os.makedirs(bd_a, exist_ok=True)

ymd = generate_ymd()
ymd_HMS = generate_ymd_HMS()
i = '(08-09)'
bd_a_img = f'{bd_a}/{ymd}'
os.makedirs(bd_a_img, exist_ok=True)

In [2]:
def get_words(g: str, n: str = 'de') -> DataFrame:
    def _get_words(w: str) -> List[dict]:
        url = ''
        headers = {
            'Host': '',
            'Origin': 'https://www.amz123.com',
            'Referer': 'https://www.amz123.com/',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
        }
        params = {
            'limit': '10',
            'suggestion-type': 'WIDGET',
            'suggestion-type': 'KEYWORD',
            'page-type': 'Gateway',
            'alias': 'aps',
            'site-variant': 'desktop',
            'version': '3',
            'event': 'onkeypress',
            'wc': '',
            'lop': '',
            'avg-ks-time': '',
            'fb': '1',
            'mid': '',
            'plain-mid': '',
            'client-info': 'amazon-search-ui',
            'prefix': w,
        }
        if n == 'de':
            url = 'https://completion.amazon.de/api/2017/suggestions'
            headers['Host'] = 'completion.amazon.de'
            params['lop'] = 'de_DE'
            params['avg-ks-time'] = '829'
            params['mid'] = 'A1PA6795UKMFR9'
            params['plain-mid'] = '4'
        elif n == 'us':
            url = 'https://completion.amazon.com/api/2017/suggestions'
            headers['Host'] = 'completion.amazon.com'
            params['lop'] = 'zh_CN'
            params['avg-ks-time'] = '0'
            params['mid'] = 'ATVPDKIKX0DER'
            params['plain-mid'] = '1'
        response = requests.get(url, headers=headers, params=params)
        try:
            result = response.json()['suggestions']
            for item in result:
                item['group'] = g
                item['parent'] = w
                item['term'] = item['value']
            return result
        except KeyError:
            pass

    ws = [i['value'] for i in _get_words(g)]

    rd = []
    for _ in ws:
        print(f'***{_}***')
        rd.extend(_get_words(_))
    df: DataFrame = pd.DataFrame(rd)
    return df[['group', 'parent', 'term']]

In [3]:
from util.widgets import random_indexes, wash_rows

def combine_words(df: DataFrame, n: int = 20, x: int = 8, shuffle: bool = True) -> DataFrame:
    indexes = random_indexes(len(df), n, x, shuffle)

    st250s = []
    count = 0

    for index in indexes:
        rows = df.iloc[index]
        s = ' '.join(rows['term'])
        st250 = {
            'group': rows['group'].values[0],
            'parent': rows['parent'].values[0],
            'len': len(s),
            'value': s,
        }
        if st250['len'] <= 250:
            st250s.append(st250)
            count += 1

    long_df: DataFrame = pd.DataFrame(st250s)
    return long_df

In [4]:
from util.widgets import generate_ym, generate_ymd, generate_i

ym = generate_ym()
g = 'cargohose_herren'
bd = f'{g}/{ym}'
b_d = f'{g}_{ym}'

os.makedirs(bd, exist_ok=True)

In [5]:
st_df = get_words(g.replace('_', ' '),n='de')

***cargohose herren***
***cargohose herren kurz***
***cargohose herren 3/4***
***cargohose herren schwarz***
***cargohose herren baggy***
***cargohose herren sommer***
***cargohose herren stretch***
***cargohose herren lang***
***cargohose herren jack and jones***
***cargohose herren kurz 3/4***


In [6]:
st_df = wash_rows(st_df)
mask = st_df['parent'] != st_df['term']
st_df = st_df[mask]
st250_df = combine_words(st_df, n=100)

In [7]:
st250_df['ymd'] = ymd

from util.widgets import Status

st250_df['status'] = Status.UNUSED.value
st250_df['ymd_HMS'] = ''

In [8]:
fp_st250 = f'{bd}/{b_d}_st250.xlsx'

st_df_ori = pd.read_excel(fp_st250, sheet_name='st')
st250_df_ori = pd.read_excel(fp_st250, sheet_name='st250')

In [9]:
st250_df_ori

,group,parent,len,value,status
0,cargohose herren,cargohose herren,240,cargohose herren baggy cargohose herren sommer...,Unused
1,cargohose herren,cargohose herren,236,cargohose herren baggy cargohose herren sommer...,Unused
2,cargohose herren,cargohose herren,230,cargohose herren stretch cargohose herren somm...,Unused
3,cargohose herren,cargohose herren baggy,241,cargohose herren baggy y2k cargohose herren st...,Unused
4,cargohose herren,cargohose herren baggy,243,cargohose herren baggy baumwolle cargohose her...,Unused
5,cargohose herren,cargohose herren baggy,239,cargohose herren baggy weiß cargohose herren s...,Unused
6,cargohose herren,cargohose herren baggy,238,cargohose herren baggy weiß cargohose herren s...,Unused
7,cargohose herren,cargohose herren lang,239,cargohose herren lang weit cargohose herren sc...,Unused
8,cargohose herren,cargohose herren lang,239,cargohose herren langgröße cargohose herren so...,Unused
9,cargohose herren,cargohose herren lang,244,cargohose herren langgröße cargohose herren sc...,Unused


In [10]:
st_df = pd.concat([st_df_ori, st_df], ignore_index=True)
st_df.drop_duplicates(subset=['term'], keep='last', inplace=True)
st_df.sort_values(by=['parent'], inplace=True)
st_df

,group,parent,term
34,cargohose herren,cargohose herren,cargohose herren schwarz
35,cargohose herren,cargohose herren,cargohose herren baggy
36,cargohose herren,cargohose herren,cargohose herren sommer
37,cargohose herren,cargohose herren,cargohose herren stretch
38,cargohose herren,cargohose herren,cargohose herren lang
9,cargohose herren,cargohose herren baggy,cargohose herren baggy blau
49,cargohose herren,cargohose herren baggy,cargohose herren baggy mit print
48,cargohose herren,cargohose herren baggy,cargohose herren baggy baumwolle
47,cargohose herren,cargohose herren baggy,cargohose herren baggy schwarz
46,cargohose herren,cargohose herren baggy,cargohose herren baggy y2k


In [11]:
st250_df = pd.concat([st250_df_ori, st250_df], ignore_index=True)
st250_df.drop_duplicates(subset=['value', 'status'], keep='last', inplace=True)
st250_df.loc[st250_df.duplicated(subset=['value']), 'status'] = Status.IN_USE.value
st250_df.drop_duplicates(subset=['value', 'status'], keep='first', inplace=True)
st250_df.sort_values(by=['parent'], inplace=True)
st250_df

,group,parent,len,value,status,ymd
0,cargohose herren,cargohose herren,240,cargohose herren baggy cargohose herren sommer...,Unused,NaN
20,cargohose herren,cargohose herren,231,cargohose herren stretch cargohose herren somm...,Unused,
19,cargohose herren,cargohose herren,244,cargohose herren schwarz cargohose herren somm...,Unused,
23,cargohose herren,cargohose herren,237,cargohose herren stretch cargohose herren schw...,Unused,
98,cargohose herren,cargohose herren,248,cargohose herren stretch cargohose herren stre...,Unused,
...,...,...,...,...,...,...
61,cargohose herren,cargohose herren stretch,246,cargohose herren stretch slim fit cargohose he...,Unused,
60,cargohose herren,cargohose herren stretch,247,cargohose herren stretch sommer cargohose herr...,Unused,
49,cargohose herren,cargohose herren stretch,245,cargohose herren stretch übergröße cargohose h...,Unused,
18,cargohose herren,cargohose herren stretch,242,cargohose herren stretch lang cargohose herren...,Unused,NaN


In [12]:
with pd.ExcelWriter(fp_st250) as writer:
    st_df.to_excel(writer, index=False, sheet_name='st')
    st250_df.to_excel(writer, index=False, sheet_name='st250')